In [ ]:
import pandas as pd
from collections import defaultdict
import json
import os

In [5]:
data_file = 'data/MQ2007/Fold1/test.txt'
output_dir = 'data/MQ2007/Fold1/test_json'

In [6]:
def get_doc_details(doc_string):
    doc_ls = doc_string.split(" ")
    return {"doc_id":doc_ls[2],"inc":doc_ls[5],"prob":doc_ls[8]}
    
def get_document_featues(feature_ls):
    d_ls = {f.split(":")[0]:f.split(":")[1] for f in feature_ls}
    return d_ls

def get_line_features(line):
    
    #data_line,doc_comments
    Z,doc_details_str = line.split("#")
    z_ls = Z.split(" ")
    y,x  = z_ls[0],z_ls[1:-1]
    doc_f = get_document_featues(x)
    doc_details_d = get_doc_details(doc_details_str)
    json_d = {"y":y, **doc_f, **doc_details_d}
    
    return json_d
def parse_file(data_file):
    query_d = defaultdict(list)
    with open(data_file) as fp:  
        line = fp.readline()
        cnt = 0 
        while line:
            strip_line = line.strip()
            q_json = get_line_features(strip_line)
            query_d[q_json['qid']].append(q_json)
            if cnt%10000==0:
                print("line cnt :{}".format(cnt))
            line = fp.readline()
            cnt+=1
            
    return query_d

def create_data_dir(doc_json,output_dir):
    """
        dumps data in the form output_dir/q_id.csv
        
    """
    os.makedirs(output_dir, exist_ok=True)
    for q_id,query in doc_json.items():
        q_df = pd.DataFrame(query)
        q_df = q_df.sort_values(by=['y'],ascending=False)
        q_df.to_csv("{}/{}.csv".format(output_dir,q_id),index=None)

def get_metafile(doc_json):
    """
        returns metafile of the doc_json
    """
    q_meta_ls = []
    for q_id,query in doc_json.items():
        q_d = {"qid":q_id,"num_docs":len(query)}
        q_meta_ls.append(q_d)
    
    return pd.DataFrame(q_meta_ls)



In [7]:
doc_json = parse_file(data_file)
create_data_dir(doc_json,output_dir)
mdf = get_metafile(doc_json)
mdf.to_csv("{}/{}.csv".format(output_dir,"metafile"),index=None)

line cnt :0
line cnt :10000


In [ ]:
#doc_json[0]

In [ ]:
#mdf.iloc[0]['qid']